In [540]:
import os
import sys


SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["PYTHONHASHSEED"] = "42"
PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [541]:
from pyspark import SparkContext, SparkConf
import sys


conf = SparkConf()
sc = SparkContext(appName="hw_3_Uhbifg_1", conf=conf)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=hw_3_Uhbifg_1, master=yarn) created by __init__ at <ipython-input-515-bbbc8b4fe7cc>:6 

In [542]:
start_node = 12 # sys.argv[1]
end_node = 34 # sys.argv[2]
path_data = "/datasets/twitter/twitter.tsv"# sys.argv[3]
path_ans = "output_1"# sys.argv[4]

In [543]:
raw_graph = sc.textFile(path_data)
graph = raw_graph.map(lambda x: map(int, x.split("\t")[::-1])).cache()
links = graph.groupByKey().partitionBy(sc.defaultParallelism).mapValues(list).cache()     


In [545]:
w = dict(links.collect())

In [546]:
def search(x1, x2, x3, seen):
    ans = list()
    a = x3[x1]
    for i in a:
        if i not in seen:
            ans.append(tuple([i, x2 + [i]]))
    return ans


links = graph.groupByKey().mapValues(list).cache()     
answers = []
pathes = sc.parallelize([(start_node, [start_node])])
seen = set([start_node])
num = 0
while True:
    if pathes.count() == 0:
        ans = sc.parallelize([])
        break
    ans = pathes.filter(lambda x : x[0] == end_node)
    if ans.count() > 0:
        break
    else:
        rdd2 = pathes.map(lambda x : x[0], preservesPartitioning=True).distinct().collect()
        neigbours = dict() 
        for i in rdd2:
            neigbours[i] = w.get(i, [])
        pathes = sc.parallelize(pathes.flatMap(lambda x : search(x[0], x[1], neigbours, seen), preservesPartitioning=True).collect())
        seen.update(pathes.map(lambda x : x[0]).distinct().collect())
ans = ans.map(lambda x : x[1])

In [547]:
ans.collect()

[[12, 14, 34], [12, 21, 34], [12, 23, 34], [12, 295, 34], [12, 322, 34]]

In [565]:
path_ans = "1.csv"

In [585]:
def toCSVLine(data):
    return ','.join(str(d) for d in data)

lines = ans.map(toCSVLine)
lines.coalesce(1).saveAsTextFile(path_ans)

In [578]:
read_rdd = sc.textFile("1.csv")


In [579]:
read_rdd.collect()

['12,295,34', '12,322,34', '12,14,34', '12,21,34', '12,23,34']

In [553]:
lines.collect()

['12,14,34', '12,21,34', '12,23,34', '12,295,34', '12,322,34']

In [557]:
!hdfs dfs -rm -rf 1.csv

-rm: Illegal option -rf
Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-copyFromLocal [-f] [-p] [-l] [-d] [-t <thread count>] <localsrc> ... <dst>]
	[-copyToLocal [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-count [-q] [-h] [-v] [-t [<storage type>]] [-u] [-x] [-e] <path> ...]
	[-cp [-f] [-p | -p[topax]] [-d] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] [-v] [-x] <path> ...]
	[-expunge]
	[-find <path> ... <expression> ...]
	[-get [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-getfacl [-R] <path>]
	[-getfattr [-R] {-n name | -d} [-e en] <path>]
	[-getmerge [-nl] [-skip-empty-file] <src> <localdst>]
	[-head <file>]
	[-help [cmd ...]]
	[-ls [-C] [-d] [-h] [-q] [

In [586]:
!hdfs dfs -ls 

Found 13 items
drwx------   - Uhbifg Uhbifg          0 2021-03-30 02:51 .Trash
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-03 14:12 .hiveJars
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-30 02:32 .sparkStaging
drwx------   - Uhbifg Uhbifg          0 2021-02-24 14:46 .staging
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-30 02:56 1.csv
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-30 00:04 1.csv1
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-30 02:30 1.txt
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-14 22:44 Uhbifg_hiveout
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-30 02:24 Uhbifg_hw3_output
drwx------   - Uhbifg Uhbifg          0 2021-02-15 10:40 hwzero
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-29 23:55 output
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-29 23:56 output_1
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-02-23 13:33 pred_with_filter


In [587]:
!hdfs dfs -cat 1.csv

cat: `1.csv': Is a directory


In [590]:
!hdfs dfs -cat 1.csv/part-00000

12,14,34
12,21,34
12,23,34
12,295,34
12,322,34


In [582]:
!hdfs dfs -rm -r -f 1.csv

21/03/30 02:55:21 INFO fs.TrashPolicyDefault: Moved: 'hdfs://name1.ru-central1.internal:8020/user/Uhbifg/1.csv' to trash at: hdfs://name1.ru-central1.internal:8020/user/Uhbifg/.Trash/Current/user/Uhbifg/1.csv1617072921217


In [589]:
!hdfs dfs -ls 1.csv

Found 2 items
-rw-r--r--   3 Uhbifg Uhbifg          0 2021-03-30 02:56 1.csv/_SUCCESS
-rw-r--r--   3 Uhbifg Uhbifg         47 2021-03-30 02:56 1.csv/part-00000
